## Creating RISK_LKP_ENTITY_THRESHOLD  Table

In [1]:
import pandas as pd,numpy as np
import pyodbc
import statistics
from configparser import ConfigParser
from scipy.stats import shapiro
from scipy.stats import kstest
from scipy import  stats as s

In [2]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
#config.read(r'config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

## Read Data and Generating Threshold 

In [6]:
TRAN_RULES = pd.read_sql_query("""SELECT  R.RULE_NO,R.ENTITY_TYPE,R.RULE_VALUE  AS RULE_VALUE,R.PEER_GROUP_ID 
                                  FROM [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_RULES (NOLOCK) R
                                  INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) 
                                  ON RTI.REPORT_ID =  R.REPORT_ID
                                  WHERE RISK_SCORE_STATIC=0""", conn)
conn.close()

In [7]:
peer_groups = TRAN_RULES['PEER_GROUP_ID'].unique()
non_stat_rules = TRAN_RULES['RULE_NO'].unique()
entity_types = ['I','O']

In [8]:
def not_normal_outliers_removal(d):
    print("not_normal_outliers_removal")
    data = [*set(d)]
    return compute_IQR(d,data)

In [9]:
def compute_IQR(d,data):
    print("compute_IQR")
    q3 = np.quantile(data, 0.75)
    q1 = np.quantile(data, 0.25)
    iqr = ((q3-q1)* 1.5)
    outliers_left = q1 - iqr
    outliers_right = q3 + iqr
    print("outliers_right",outliers_right)
#     data = list(filter(lambda x: outliers_left <= x <= outliers_right, data))
    data = list(filter(lambda x: x <= outliers_right,d))

    return data,outliers_right

In [10]:
def normal_outliers_removal(d):
    print("normal_outliers_removal")
    mean = np.mean(d)
    std = np.std(d)
    outliers_right = (mean + (3*std))
    data = list(filter(lambda x: x <= outliers_right, d))
    print(outliers_right)
    return data,outliers_right

In [11]:
def thresh_cal(d,outliers_right):
    print("thresh")
    median = np.median(d)
    mean = np.mean(d)
    std = np.std(d)
    mode = int(s.mode(d)[0])
    
    if std >= 1:
        one = round(median+std)
        two = round(median+2*std)
        three = round(median+3*std)
        four = round(median+4*std)
        five = round(median+5*std)
        six = round(median+6*std)
        seven = round(median+7*std)
        eight = round(median+8*std)
        nine =round(median+9*std)
        ten = round(median+10*std)

    elif std > 0:
        one = np.ceil(median+std)
        two = np.ceil(one+std)
        three = np.ceil(two+std)
        four = np.ceil(three+std)
        five = np.ceil(four+std)
        six = np.ceil(five+std)
        seven = np.ceil(six+std)
        eight = np.ceil(seven+std)
        nine =np.ceil(eight+std)
        ten = np.ceil(nine+std)
        
        
    elif std == 0:
        one = np.ceil(median+std+1)
        two = np.ceil(one+std+1)
        three = np.ceil(two+std+1)
        four = np.ceil(three+std+1)
        five = np.ceil(four+std+1)
        six = np.ceil(five+std+1)
        seven = np.ceil(six+std+1)
        eight = np.ceil(seven+std+1)
        nine =np.ceil(eight+std+1)
        ten = np.ceil(nine+std+1)

        
    print(one,two,three,four,five,six,seven,eight,nine,ten,mean,median,mode,std,outliers_right)

    return(one,two,three,four,five,six,seven,eight,nine,ten,mean,median,mode,std,outliers_right)

In [12]:
def compute_outlier(d):
    try:    
        if (len(d) == 0):
            print("Not applicable")
            return( None,None,None,None,None,None,None,None,None,None,None,None,None,None,None) 
        elif ( len(d) < 5):
            print("Not applicable")
            return thresh_cal(d,"NA") 
        else:
            if ( len(d) <= 150):
                stats,p_val = shapiro(d)
                if p_val > 0.05:
                    d,outliers_right = normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)
                else:
                    d,outliers_right = not_normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)          
            else:
                stats,p_val  = kstest(d, 'norm')
                if p_val > 0.05:
                    d,outliers_right = normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)
                    
                else:
                    d,outliers_right = not_normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)      
    except KeyError:
        return( None,None,None,None,None,None,None,None,None,None,None,None,None,None,None)     
    return( None,None,None,None,None,None,None,None,None,None,None,None,None,None,None)

In [13]:
peer_th = []
for peer_group in  peer_groups:
    temp = TRAN_RULES[TRAN_RULES['PEER_GROUP_ID'] == peer_group]
    for entity_type in entity_types:
        temp2 = temp[temp['ENTITY_TYPE'] == entity_type]
        for rule_no in non_stat_rules:
            data = temp2[temp2['RULE_NO'] == rule_no]
            data = data['RULE_VALUE'].astype('float64')
            data = list(map(int,data))
            if(len(data) == 0):
                maximum = 'null'
                minimum = 'null'
            else:
                maximum = max(data)
                minimum = min(data)
            one,two,three,four,five,six,seven,eight,nine,ten,mean,median,mode,std,outliers_right = compute_outlier(data)
            l =[peer_group,entity_type,rule_no,one,two,three,four,five,six,seven,eight,nine,ten,mean,median,mode,std,outliers_right,maximum,minimum]
            peer_th.append(l)

Not applicable
thresh
5000001.0 5000002.0 5000003.0 5000004.0 5000005.0 5000006.0 5000007.0 5000008.0 5000009.0 5000010.0 5000000.0 5000000.0 5000000 0.0 NA
Not applicable
thresh
4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 13.0 3.0 3.0 3 0.0 NA
Not applicable
thresh
4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 13.0 3.0 3.0 3 0.0 NA
not_normal_outliers_removal
compute_IQR
outliers_right 1200001.25
thresh
706156 912311 1118467 1324623 1530778 1736934 1943090 2149245 2355401 2561556 550000.1666666666 500000.0 500000 206155.64508433643 1200001.25
Not applicable
not_normal_outliers_removal
compute_IQR
outliers_right 2.5
thresh
3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 1.6666666666666667 2.0 2 0.47140452079103173 2.5
Not applicable
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.5 1.5 1 0.5 NA
Not applicable
Not applicable
thresh
740350596592 1110519356560 1480688116528 1850856876496 2221025636464 2591194396432 2961363156400 3331531916368 3701700676336 4071869436304 370181836624.0 37

Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
thresh
1522001.0 1522002.0 1522003.0 1522004.0 1522005.0 1522006.0 1522007.0 1522008.0 1522009.0 1522010.0 1522000.0 1522000.0 1522000 0.0 NA
Not applicable
thresh
4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 13.0 3.0 3.0 3 0.0 NA
Not applicable
thresh
4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 13.0 3.0 3.0 3 0.0 NA
Not applicable
Not applicable
thresh
13853813 17707626 21561439 25415252 29269065 33122878 36976691 40830504 44684317 48538130 7775000.0 10000000.0 10000000 3853813.046840752 NA
Not applicable
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.5 1.5 1 0.5 NA
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 0 0.816496580927726 NA
Not applicable
Not applicable
thresh
1500000 2210000 2920000 3630000 4340000 505000

/opt/conda/lib/python3.6/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


500000 700000 900000 1100000 1300000 1500000 1700000 1900000 2100000 2300000 300000.0 300000.0 100000 200000.0 NA
Not applicable
not_normal_outliers_removal
compute_IQR
outliers_right 16808651.125
thresh
3360780 6548215 9735650 12923085 16110520 19297955 22485390 25672825 28860260 32047695 1553692.6764705882 173344.5 50000 3187435.032950362 16808651.125
Not applicable
thresh
177777778.0 177777779.0 177777780.0 177777781.0 177777782.0 177777783.0 177777784.0 177777785.0 177777786.0 177777787.0 177777777.0 177777777.0 177777777 0.0 NA
Not applicable
not_normal_outliers_removal
compute_IQR
outliers_right 2.5
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.05 1.0 1 0.21794494717703367 2.5
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 1 0.0 NA
not_normal_outliers_removal
compute_IQR
outliers_right 4.5
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.1538461538461537 1.0 1 0.532938710021193 4.5
Not applicable
Not applicable
Not applicable
Not applicable
Not app

Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 1 0.0 NA
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not 

In [14]:
thresh = pd.DataFrame(data = peer_th,columns = ['PEER_GROUP_ID','ENTITY_TYPE','RULE_NO','ONE','TWO','THREE','FOUR','FIVE','SIX',
                                                'SEVEN','EIGHT','NINE','TEN','MEAN','MEDIAN','MODE','STD','OUTLIER', 
                                                'maximum','minimum'])

thresh.fillna('NULL',inplace = True)

In [15]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [18]:


# INSERTING DATA IN THRESHOLD TABLE.
for ind,row in thresh.iterrows():
    cursor.execute("""INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_ENTITY_THRESHOLD (
                      THRESHOLD1_1,THRESHOLD1_2,THRESHOLD1_3,THRESHOLD1_4,THRESHOLD1_5,THRESHOLD1_6,THRESHOLD1_7,
                      THRESHOLD1_8,THRESHOLD1_9,THRESHOLD1_10,MEAN,MEDIAN,MODE,
                      STANDARD_DEVIATION,OUTLIER,PEER_GROUP_ID,ENTITY_TYPE,RULE_NO,MAX,MIN) 
                      VALUES({},{},{},{},{},{},{},{},{},{},{},{},{},{},'{}','{}','{}','{}',{},{} )"""
                      .format(row['ONE'],row['TWO'],row['THREE'],row['FOUR'],row['FIVE'],row['SIX'],row['SEVEN'],row['EIGHT'],
                      row['NINE'],row['TEN'],row['MEAN'],row['MEDIAN'],row['MODE'],row['STD'],row['OUTLIER'],
                      row['PEER_GROUP_ID'],row['ENTITY_TYPE'],row['RULE_NO'],row['maximum'],row['minimum']))
    conn.commit()


In [19]:
print('Threshold_complete')

Threshold_complete
